In [1]:
# Install dependencies
#!pip install PyExecJS
#!pip install --upgrade mplfinance
#!pip install jupyter-require
#!pip install ipywidgets

In [2]:
# Import useful modules
import ipywidgets as widgets
from IPython.display import display, HTML, Javascript
display(HTML("<style>.container { width:100% !important; }</style>"))

from datetime import datetime, date, timedelta
import matplotlib.pyplot as plt
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()
import pandas as pd

from jesse import research, utils
import jesse.indicators as ta
import custom_indicators as cta
from custom_indicators import ma
import numpy as np

from mplfinance.original_flavor import candlestick_ohlc
from matplotlib.dates import HourLocator, MonthLocator, YearLocator

import matplotlib.dates as mdates
import matplotlib.ticker as mticker

import json

import lightweight as lw

In [3]:
# Useful format functions
def lines_to_df(cols_array, data_series, time_series, col_time = "time"):
    df = pd.DataFrame(data=data_series, index=time_series, columns=cols_array)
    df[col_time] = pd.to_datetime(df.index, unit="ms")
    return df

def pct_to_decimal(nb, pct: float = 0.01):
    try: 
        return int(np.log10(abs(nb) * pct / 100))
    except:
        return 8

def floor_by_pct(nb, pct: float = 0.01):
    return round(nb, -pct_to_decimal(nb, pct))

def to_datetime(timestamp):
    return datetime.fromtimestamp(float(timestamp)/1000)

def to_date(timestamp):
    return datetime.fromtimestamp(float(timestamp)/1000).strftime("%Y-%m-%d %H:%M")

def format_time(seconds):
    if seconds < 60:
        return f"{seconds} second{'s' if seconds != 1 else ''}"
    elif seconds < 3600:
        minutes = int(seconds // 60)
        return f"{minutes} minute{'s' if minutes != 1 else ''}"
    elif seconds < 86400:
        hours = int(seconds // 3600)
        return f"{hours} hour{'s' if hours != 1 else ''}"
    else:
        days = int(seconds // 86400)
        return f"{days} day{'s' if days != 1 else ''}"

def timeframe_to_duration(timeframe):
    letters = {'m': 1, 'h': 60, 'D': 3600, 'W': 25200, 'M': 100800}
    return int(timeframe[:-1])*letters[timeframe[-1]]

def truncate_to_timestep(dt, timestep_minutes, bias=0):
    if isinstance(dt, int) or isinstance(dt, float) or isinstance(dt, str):
        dt = to_datetime(dt)
    # Convert timestep to seconds
    timestep_seconds = timestep_minutes * 60
    # Calculate the number of seconds since midnight
    seconds_since_midnight = (dt - dt.replace(hour=0, minute=0, second=0, microsecond=0)).total_seconds()
    # Round down to the nearest timestep
    truncated_seconds = seconds_since_midnight if seconds_since_midnight % timestep_seconds == 0 else (seconds_since_midnight // timestep_seconds + bias) * timestep_seconds 
    # Create a new datetime object with the truncated time
    truncated_dt = dt.replace(hour=0, minute=0, second=0, microsecond=0) + timedelta(seconds=truncated_seconds)
    return truncated_dt


In [4]:
# Select files and parameters
name_file = 'TrendSwingTrader_BF_BTC-USDT_4h_2021-08-01_2024-08-01_0675'

with open(f'storage/json/{name_file}.json', 'r') as file:
    bt = json.load(file)

with open(f'storage/data/data_{name_file}.json', 'r') as file:
    data_series = json.load(file)

timeframe = '4h'
warmup = 210
balance = 10000

In [5]:
# Print the backtest info
timestep = timeframe_to_duration(timeframe)
timestamp_start = float(bt['trades'][0]['opened_at'])
timestamp_end = float(bt['trades'][-1]['closed_at'])
for serie in data_series:
    timestamp_start = min(timestamp_start, float(list(data_series[serie].keys())[0]))
    timestamp_end = max(timestamp_end, float(list(data_series[serie].keys())[-1]))
warmup_start = timestamp_start - warmup * timestep * 1000 * 60

date_warmup_start = to_datetime(warmup_start)
date_start = to_datetime(timestamp_start)
date_end = to_datetime(timestamp_end)
days_range = (date_end-date_start).days

# Window time interval
exchange = bt['trades'][0]['exchange']
symbol = bt['trades'][0]['symbol']

print(f'warmup_start: {date_warmup_start}')
print(f'date_start: {date_start}')
print(f'date_end: {date_end}')
print(f'exchange: {exchange}')
print(f'symbol: {symbol}')
print(f'days range: {days_range}')

# Getting candles data from Exchange
candles = research.get_candles(exchange, symbol, timeframe, warmup_start, timestamp_end)[-1]

warmup_start: 2021-06-27 04:00:00
date_start: 2021-08-01 04:00:00
date_end: 2024-08-01 00:00:00
exchange: Binance Perpetual Futures
symbol: BTC-USDT
days range: 1095


In [6]:
# Build buy orders and sell orders time series
data_orders = []
for trade in bt['trades']:
    data_orders += trade['orders'] 

buys = {}
sells = {}
for order in data_orders:
    o = order["__data__"]
    time = truncate_to_timestep(to_datetime(o['executed_at']), timestep)
    if o['side'] == 'buy':
        buys[time] = o['price']
    else:
        sells[time] = o['price']

buys = pd.Series(buys)
sells = pd.Series(sells)

In [7]:
# Build balance and exposure time series
balance = 10000
avg_entry = np.nan
pos_qty = 0
side = 0
balances = {date_start: balance}
exposures = {date_start: 0}
avg_entries = {date_start: np.nan}

for trade in bt['trades']:
    avg_entry = 0
    pos_qty = 0
    side = trade['orders'][0]['__data__']['side']
    for order in trade['orders']:
        o = order['__data__']
        time = truncate_to_timestep(to_datetime(o['executed_at']), timestep)
        if side == o['side']:
            avg_entry, pos_qty = (avg_entry*pos_qty + o['price']*o['qty'])/(pos_qty + o['qty']), pos_qty + o['qty']
            o['pnl'] = 0
            o['pnl_pct'] = 0
        else:
            avg_entry, pos_qty = avg_entry, pos_qty + o['qty']
            pnl = (avg_entry - o['price']) * o['qty']
            o['pnl'] = pnl
            o['pnl_pct'] = pnl/balance*100
            balance += pnl
            balances[time] = balance
        exposures[time] = pos_qty
        avg_entries[time] = avg_entry
    balance -= trade['fee']
    balances[time] = balance

balances = pd.Series(balances)
exposures = pd.Series(exposures)
avg_entries = pd.Series(avg_entries)

In [8]:
# Build margin and equity series
margins = {}
equities = {}

margin = balances[date_start]
exposure = 0
avg_entry = 0
for candle in candles[warmup:]:
    time = to_datetime(candle[0])
    if time in exposures:
        if time in balances: margin = balances[time]
        exposure = exposures[time]
        avg_entry = avg_entries[time]
    margins[time] = margin
    equities[time] = margin + (candle[2] - avg_entry)*exposure

margins = pd.Series(margins)
equities = pd.Series(equities)

In [9]:
# Plot equity and balance on the same chart
lw.plot_series(
    {'Equity': equities}, 
    config = [{'name': 'Equity', 'attach': True, 'fn': 'addLineSeries', 'style': {'color': '#CC0BA1', 'lineWidth': 2}},],
    width=700, 
    height=300, 
    dark_theme = True,
    chart_name='equity')

<IPython.core.display.Javascript object>

In [10]:
# Display the HTML interactive trade table
html_table = """
    <style>
        .clickable-row {
            cursor: pointer;
            display: table-row;
        }
                
        .clickable-row:hover {
            background-color: #606060;
        }
                
        table  {
            border-spacing: 1; 
            border-collapse: collapse; 
            border-radius: 6px;
            position: relative; 
            text-align: right;
            margin-left: auto;
            margin-right: auto;
        }
          
        td,th  { padding-left: 8px }
        
        thead tr  {
            height: 40px;
            font-size: 16px;
        }
        
        .rTable {
            display: table;
            text-align: right;
        }
        .rTableRow {
            display: table-row;
        }
        .rTableHeading {
            display: table-header-group;
            background-color: #333333;
            height: 40px;
            font-weight: bold;
            font-size: 16px;
        }
        .rTableHead {
            display: table-cell;
            padding: 10px 10px;
            border: 1px solid #999999;
        }
        .rTableCell {
            display: table-cell;
            padding: 2px 10px;
            border: 1px solid #999999;
        }
    </style>

    <div class="rTable">
        <div class="rTableHeading">
            <div class="rTableRow">
                <div class="rTableHead" style="width: 5%; min-width: 40px">Type</div>
                <div class="rTableHead" style="width: 10%; min-width: 90px">Entry price</div>
                <div class="rTableHead" style="width: 10%; min-width: 90px">Exit price</div>
                <div class="rTableHead" style="width: 10%; min-width: 60px">PNL</div>
                <div class="rTableHead" style="width: 10%; min-width: 60px">PNL %</div>
                <div class="rTableHead" style="width: 5%; min-width: 50px">Fee</div>
                <div class="rTableHead" style="width: 15%; min-width: 110px">Opened at</div>
                <div class="rTableHead" style="width: 15%; min-width: 110px">Closed at</div>
                <div class="rTableHead" style="width: 10%; min-width: 70px">Duration</div>
            </div>
        </div>
    </div>"""

# Trades and orders table
for index, trade in enumerate(bt['trades']):
    html_table += f"""
                <div class="rTable">
                    <div class="clickable-row" id="trade{index}">
                        <div class="rTableCell" style="width: 5%; min-width: 40px" id="type">{trade['type']}</div>
                        <div class="rTableCell" style="width: 10%; min-width: 90px" id="entry">{floor_by_pct(trade['entry_price'])}</div>
                        <div class="rTableCell" style="width: 10%; min-width: 90px" id="exit">{floor_by_pct(trade['exit_price'])}</div>
                        <div class="rTableCell" style="width: 10%; min-width: 60px" id="pnl"><font style="color: {'#30ff30' if float(trade['PNL']) > 0 else '#ff0000'}">{floor_by_pct(trade['PNL'])}</font></div>
                        <div class="rTableCell" style="width: 10%; min-width: 60px" id="pnl_pct"><font style="color: {'#30ff30' if float(trade['PNL_percentage']) > 0 else '#ff0000'}">{floor_by_pct(trade['PNL_percentage'])} %</font></div>
                        <div class="rTableCell" style="width: 5%; min-width: 50px" id="fee">{np.round(trade['fee'],2)}</div>
                        <div class="rTableCell" style="width: 15%; min-width: 110px" id="date">{to_date(trade['opened_at'])}</div>
                        <div class="rTableCell" style="width: 15%; min-width: 110px">{to_date(trade['closed_at'])}</div>
                        <div class="rTableCell" style="width: 10%; min-width: 70px">{format_time(trade['holding_period'])}</div>
                    </div>
                </div>
                <div id="orders{index}" style="display: none; min-width: 520px;">
                    <table>
                        <thead>
                            <tr>
                                <th>Side</th>
                                <th>Type</th>
                                <th>Quantity</th>
                                <th>Price</th>
                                <th>PNL</th>
                                <th>PNL </th>
                                <th>Created at</th>
                                <th>Executed at</th>
                            </tr>
                        </thead>
                        <tbody>"""
    
    for i, order in enumerate(trade['orders']):
        html_table += f"""
                        <tr id="order">
                            <td id="side">{order["__data__"]['side']}</td>
                            <td id="type">{order["__data__"]['type']}</td>
                            <td id="qty">{floor_by_pct(order["__data__"]['qty'])}</td>
                            <td id="price">{floor_by_pct(order["__data__"]['price'])}</td>
                            <td id="pnl"><font style="color: {'#30ff30' if order["__data__"]['pnl'] > 0 else '#ff0000' if order["__data__"]['pnl'] < 0 else ''}">{np.round(order["__data__"]['pnl'],2)}</font></td>
                            <td id="pnl_pct"><font style="color: {'#30ff30' if order["__data__"]['pnl_pct'] > 0 else '#ff0000'  if order["__data__"]['pnl'] < 0 else ''}">{np.round(order["__data__"]['pnl_pct'],2)} %</font></td>
                            <td id="created">{datetime.fromtimestamp(order["__data__"]['created_at']/1000).strftime("%Y-%m-%d %H:%M")}</td>
                            <td id="date">{datetime.fromtimestamp(order["__data__"]['executed_at']/1000).strftime("%Y-%m-%d %H:%M")}</td>
                        </tr>
                            """
    html_table += f"""
                        </tbody>
                    </table>
                </div>"""

display(HTML(html_table))

# Click on a trade to toggle the orders table display
display(Javascript("""document.querySelectorAll('.clickable-row').forEach(row => {
    row.addEventListener('click', () => {
        const detailsRow = row.parentElement.nextElementSibling;
        if (detailsRow.style.display === 'none' || detailsRow.style.display === '') {
            detailsRow.style.display = 'inline-block';
        } else {
            detailsRow.style.display = 'none';
        }
    });
});"""))

Side,Type,Quantity,Price,PNL,PNL,Created at,Executed at
buy,MARKET,0.2352,42421.0,0,0 %,2021-08-01 04:00,2021-08-01 04:00
sell,STOP,-0.2352,40488.0,-454.56,-4.55 %,2021-08-01 04:00,2021-08-01 22:31
Side,Type,Quantity,Price,PNL,PNL,Created at,Executed at
sell,MARKET,-0.2055,46290.0,0,0 %,2021-09-09 12:00,2021-09-09 12:00
buy,STOP,0.2055,48527.0,-459.87,-4.82 %,2021-09-09 12:00,2021-09-18 05:48
Side,Type,Quantity,Price,PNL,PNL,Created at,Executed at
sell,MARKET,-0.2101,43066.0,0,0 %,2021-09-21 16:00,2021-09-21 16:00
buy,STOP,0.2101,45205.0,-449.28,-4.95 %,2021-09-21 16:00,2021-10-01 10:28
Side,Type,Quantity,Price,PNL,PNL,Created at,Executed at
buy,MARKET,0.1724,49850.0,0,0 %,2021-10-05 12:00,2021-10-05 12:00


<IPython.core.display.Javascript object>

In [11]:
# Plot the candle chart with your indicators

# format candles
df_candles = utils.numpy_candles_to_dataframe(candles)
df_candles["time"] = pd.to_datetime(df_candles["date"]).astype(int) // 10**9

# import indicators from json
df = {}
for serie in data_series:
    df[serie] = pd.Series(dict((to_datetime(candle[0]), np.nan) if str(candle[0]) not in data_series[serie] else (to_datetime(candle[0]), data_series[serie][str(candle[0])]) for candle in candles))

# calculate indicators from candles here
times = [to_datetime(c[0]) for c in candles]
ema = ta.ema(candles, 50, sequential=True)
df['ema'] = pd.Series(data=ema, index=times)

# indicators style
config = [{'name': 'adx', 'attach': False, 'fn': 'addLineSeries', 'height': 70, 'style': {'color': '#01A781', 'lineWidth': 1}},
          {'name': 'ema', 'attach': True, 'fn': 'addLineSeries', 'style': {'color': '#F00FFF', 'lineWidth': 2}},]

lw.plot(
    df_candles,
    buys,
    sells,
    data=df,
    config=config,
    dark_theme=True, # optional
    width=1000, height=300, # optional
    chart_name='chart1'
)


<IPython.core.display.Javascript object>